In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'suprise']
labels = ['happy', 'sad']
img_size = 64
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in tqdm(os.listdir(path)):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
images_data = get_data("gdrive/My Drive/emotions_data")

100%|██████████| 6078/6078 [24:16<00:00,  4.17it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [6]:
X = []
y = []
for feature, label in images_data:
  X.append(feature)
  y.append(label)

In [7]:
X = np.array(X) / 255

In [8]:
X.reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [11]:
X_train.shape

(11300, 64, 64, 3)

In [12]:
X_test.shape

(3767, 64, 64, 3)

In [26]:
X_train[0]

array([[[0.03921569, 0.03921569, 0.03921569],
        [0.03529412, 0.03529412, 0.03529412],
        [0.03529412, 0.03529412, 0.03529412],
        ...,
        [0.07058824, 0.07058824, 0.07058824],
        [0.27843137, 0.27843137, 0.27843137],
        [0.57647059, 0.57647059, 0.57647059]],

       [[0.02352941, 0.02352941, 0.02352941],
        [0.03529412, 0.03529412, 0.03529412],
        [0.03921569, 0.03921569, 0.03921569],
        ...,
        [0.07058824, 0.07058824, 0.07058824],
        [0.25882353, 0.25882353, 0.25882353],
        [0.5372549 , 0.5372549 , 0.5372549 ]],

       [[0.02352941, 0.02352941, 0.02352941],
        [0.03921569, 0.03921569, 0.03921569],
        [0.04313725, 0.04313725, 0.04313725],
        ...,
        [0.06666667, 0.06666667, 0.06666667],
        [0.23529412, 0.23529412, 0.23529412],
        [0.48235294, 0.48235294, 0.48235294]],

       ...,

       [[0.06666667, 0.06666667, 0.06666667],
        [0.08235294, 0.08235294, 0.08235294],
        [0.0745098 , 0

In [13]:
y_train.shape

(11300,)

In [14]:
y_test.shape

(3767,)

In [29]:
y_train[0]

0

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [22]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(64,64,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 64)         

In [35]:
model.compile(optimizer = "adam" , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [36]:
history = model.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=50, batch_size=32,
                   verbose=2)

Epoch 1/50
354/354 - 62s - loss: 0.6209 - accuracy: 0.6541 - val_loss: 0.5335 - val_accuracy: 0.7306
Epoch 2/50
354/354 - 61s - loss: 0.5133 - accuracy: 0.7432 - val_loss: 0.5041 - val_accuracy: 0.7608
Epoch 3/50
354/354 - 61s - loss: 0.4638 - accuracy: 0.7716 - val_loss: 0.4334 - val_accuracy: 0.7903
Epoch 4/50
354/354 - 61s - loss: 0.4176 - accuracy: 0.7997 - val_loss: 0.3964 - val_accuracy: 0.8097
Epoch 5/50
354/354 - 61s - loss: 0.3807 - accuracy: 0.8199 - val_loss: 0.4202 - val_accuracy: 0.7993
Epoch 6/50
354/354 - 61s - loss: 0.3593 - accuracy: 0.8353 - val_loss: 0.3833 - val_accuracy: 0.8174
Epoch 7/50
354/354 - 61s - loss: 0.3255 - accuracy: 0.8519 - val_loss: 0.3561 - val_accuracy: 0.8370
Epoch 8/50
354/354 - 61s - loss: 0.3019 - accuracy: 0.8596 - val_loss: 0.3357 - val_accuracy: 0.8511
Epoch 9/50
354/354 - 61s - loss: 0.2769 - accuracy: 0.8812 - val_loss: 0.3431 - val_accuracy: 0.8466
Epoch 10/50
354/354 - 61s - loss: 0.2531 - accuracy: 0.8918 - val_loss: 0.3552 - val_accura

In [37]:
model.save("gdrive/My Drive/emotions_data/model.h5")